In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, Activation
from keras import regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import numpy as np


from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix, hstack

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Load the datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

lengthTestData = len(test_data)

In [3]:
# Combine text data for preprocessing
text = pd.concat([train_data['text'], val_data['text']], ignore_index=True)
text_test = test_data['text']

In [4]:
linguistic_features = train_data.iloc[:,-9:]
val_linguistic_features = val_data.iloc[:,-9:]
test_linguistic_features = test_data.iloc[:,-9:]

In [5]:
linguistic_features

,Words_per_Sentence,Percentage_Questions,Percentage_First_Person_Singular,Percentage_Second_Person,Percentage_Third_Person,Percentage_Negation,Percentage_Exclusive,Percentage_Causation,Percentage_Sense
0,28.777778,0.000000,0.000000,0.000000,1.930502,0.000000,0.000000,2.702703,5.405405
1,32.900000,0.000000,0.000000,0.000000,2.735562,0.607903,1.519757,1.519757,4.863222
2,30.416667,0.000000,0.000000,0.000000,2.191781,0.547945,0.821918,1.095890,6.027397
3,21.254545,3.636364,0.085543,0.000000,4.790419,1.197605,1.710864,2.138580,3.079555
4,24.444444,0.000000,0.000000,0.090909,3.181818,0.545455,1.636364,2.181818,6.181818
...,...,...,...,...,...,...,...,...,...
48634,17.466667,0.000000,1.908397,0.763359,3.053435,0.381679,1.145038,0.381679,3.053435
48635,23.500000,0.000000,3.723404,2.127660,1.063830,0.531915,0.531915,1.063830,1.063830
48636,25.785714,0.000000,0.000000,0.000000,2.908587,0.415512,1.800554,0.969529,0.831025
48637,26.809524,4.761905,0.532860,0.177620,8.703375,0.355240,1.598579,1.598579,2.131439


In [6]:
val_linguistic_features

,Words_per_Sentence,Percentage_Questions,Percentage_First_Person_Singular,Percentage_Second_Person,Percentage_Third_Person,Percentage_Negation,Percentage_Exclusive,Percentage_Causation,Percentage_Sense
0,28.866667,0.000000,0.000000,0.000000,2.078522,0.000000,0.461894,2.078522,1.154734
1,15.666667,0.000000,4.787234,0.000000,5.851064,1.063830,1.595745,0.000000,0.000000
2,24.709677,0.000000,0.000000,0.261097,1.958225,1.174935,2.349869,1.044386,6.527415
3,25.076923,0.000000,1.380368,0.153374,4.447853,0.613497,1.380368,1.533742,5.674847
4,25.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,2.000000,2.000000
...,...,...,...,...,...,...,...,...,...
6075,56.166667,16.666667,0.296736,0.000000,4.154303,1.186944,1.186944,0.593472,5.341246
6076,22.900000,0.000000,0.000000,0.000000,2.183406,0.436681,0.873362,0.000000,5.240175
6077,18.095238,14.285714,1.842105,3.684211,5.263158,1.052632,2.105263,1.315789,5.789474
6078,16.611354,10.043668,1.156677,2.576236,2.970557,1.445846,2.602524,1.393270,2.628812


In [7]:
test_linguistic_features

,Words_per_Sentence,Percentage_Questions,Percentage_First_Person_Singular,Percentage_Second_Person,Percentage_Third_Person,Percentage_Negation,Percentage_Exclusive,Percentage_Causation,Percentage_Sense
0,25.066667,0.000000,0.000000,0.265957,2.393617,0.000000,1.595745,2.393617,7.446809
1,23.333333,0.000000,0.000000,0.000000,2.857143,2.857143,2.857143,2.142857,2.857143
2,24.333333,0.000000,0.000000,0.000000,1.369863,1.369863,1.369863,0.000000,8.219178
3,27.466667,0.000000,0.404531,0.242718,3.559871,0.485437,0.566343,1.779935,3.478964
4,40.666667,0.000000,0.000000,0.000000,3.278689,0.000000,0.819672,3.278689,6.557377
...,...,...,...,...,...,...,...,...,...
6076,26.708333,8.333333,0.468019,0.624025,4.680187,0.936037,2.184087,1.560062,1.872075
6077,32.700000,0.000000,0.000000,0.000000,1.834862,0.000000,0.305810,0.000000,4.587156
6078,25.600000,0.000000,0.000000,0.000000,4.427083,0.520833,0.781250,3.125000,2.083333
6079,63.000000,0.000000,0.000000,0.000000,5.555556,0.000000,2.777778,0.793651,2.380952


In [8]:
# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [9]:
# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercase
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(filtered_tokens)

In [10]:
text_preprocessed = text.apply(preprocess_text)

In [11]:
print(text_preprocessed)

0        beirut ( reuters ) - iran military chief met s...
1        hanoi ( reuters ) - top u.s. envoy began two-d...
2        ( reuters ) - four u.s. senator asked senate j...
3        first read morning briefing meet press nbc pol...
4        cairo ( reuters ) - six month egypt election ,...
                               ...                        
54714    lack oversight prof donald trump totally unfit...
54715    tucker carlson responded espn anchor calling p...
54716    getting something nothing rage president profe...
54717    black emanuelle fixed 1976. attila speaking eu...
54718    chaos broke legal american illegal alien clash...
Name: text, Length: 54719, dtype: object


In [12]:
print(text)

0        beirut (reuters) - iran s military chief met w...
1        hanoi (reuters) - a top u.s. envoy began a two...
2        (reuters) - four u.s. senators have asked the ...
3        first read is a morning briefing from meet the...
4        cairo (reuters) - six months before egypt s el...
                               ...                        
54714    this lack of oversight proves that donald trum...
54715    tucker carlson responded to an espn anchor cal...
54716    because getting something for nothing is all t...
54717    black emanuelle fixed all that in 1976. attila...
54718    chaos broke out after legal americans and ille...
Name: text, Length: 54719, dtype: object


In [13]:
# Initialize TF-IDF vectorizer without specifying max_features
tfidf_vectorizer = TfidfVectorizer()

In [14]:
# Fit and transform the preprocessed text data
tfidf_matrix = tfidf_vectorizer.fit_transform(text_preprocessed)

In [15]:
# Get the number of unique tokens
num_unique_tokens = len(tfidf_vectorizer.get_feature_names_out())
print(num_unique_tokens)

169079


In [16]:
# Re-initialize TF-IDF vectorizer with the determined max_features
tfidf_vectorizer = TfidfVectorizer(max_features=num_unique_tokens)

In [17]:
#Fit and transform the text data again with the updated max_features
tfidf_matrix = tfidf_vectorizer.fit_transform(text_preprocessed)

# Convert the TF-IDF matrix to a CSR (Compressed Sparse Row) matrix for efficient row-wise operations
csr_tfidf_matrix = csr_matrix(tfidf_matrix)

# Find the row index with the maximum number of filled values
max_features_row_index = csr_tfidf_matrix.getnnz(axis=1).argmax()

# Get the number of features in the document with the most filled values
max_features = csr_tfidf_matrix[max_features_row_index].count_nonzero()

print(max_features)

svd = TruncatedSVD(n_components=int(max_features*0.3))
tfidf_matrix = svd.fit_transform(tfidf_matrix)

3498


In [18]:
dense_tfidf_matrix = tfidf_matrix[:len(train_data)]
dense_tfidf_with_linguistic = hstack([dense_tfidf_matrix, csr_matrix(linguistic_features)])

dense_val_tfidf_matrix = tfidf_matrix[len(train_data):len(train_data) + len(val_data)]
dense_val_tfidf_with_linguistic = hstack([dense_val_tfidf_matrix, csr_matrix(val_linguistic_features)])

In [19]:
#Define the CNN + Bi-LSTM model
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(dense_tfidf_with_linguistic.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.3))
model.add(Dense(16))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

#Define the Bi-LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(tfidf_matrix.shape[1],1)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [20]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Assuming train_data['label'] and val_data['label'] are Pandas Series, convert them to arrays
train_labels = train_data['label'].values
val_labels = val_data['label'].values

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

# Reshape the input data
dense_tfidf_with_linguistic = dense_tfidf_with_linguistic.toarray()  # Convert to dense array
dense_val_tfidf_with_linguistic = dense_val_tfidf_with_linguistic.toarray()  # Convert to dense array

# Fit the model using NumPy arrays with callbacks
model.fit(dense_tfidf_with_linguistic, train_labels, 
          epochs=100, batch_size=32, 
          validation_data=(dense_val_tfidf_with_linguistic, val_labels),
          callbacks=[early_stopping, reduce_lr])

Epoch 1/100
1520/1520 [==============================] - 488s 316ms/step - loss: 0.4190 - accuracy: 0.8085 - val_loss: 0.2904 - val_accuracy: 0.8799 - lr: 0.0010
Epoch 2/100
1520/1520 [==============================] - 421s 277ms/step - loss: 0.3119 - accuracy: 0.8703 - val_loss: 0.2756 - val_accuracy: 0.8895 - lr: 0.0010
Epoch 3/100
1520/1520 [==============================] - 428s 282ms/step - loss: 0.2940 - accuracy: 0.8774 - val_loss: 0.2596 - val_accuracy: 0.8913 - lr: 0.0010
Epoch 4/100
1520/1520 [==============================] - 631s 415ms/step - loss: 0.2754 - accuracy: 0.8872 - val_loss: 0.2525 - val_accuracy: 0.8938 - lr: 0.0010
Epoch 5/100
1520/1520 [==============================] - 628s 413ms/step - loss: 0.2678 - accuracy: 0.8894 - val_loss: 0.2659 - val_accuracy: 0.8910 - lr: 0.0010
Epoch 6/100
1520/1520 [==============================] - 608s 400ms/step - loss: 0.2599 - accuracy: 0.8929 - val_loss: 0.2479 - val_accuracy: 0.8992 - lr: 0.0010
Epoch 7/100
1520/1520 [=====

In [23]:
text_test_preprocessed = text_test.apply(preprocess_text)
test_tfidf_matrix = tfidf_vectorizer.transform(text_test_preprocessed)
dense_test_tfidf_matrix = svd.transform(test_tfidf_matrix)
dense_test_tfidf_with_linguistic = hstack([dense_test_tfidf_matrix, csr_matrix(test_linguistic_features)])
test_labels = test_data['label'].values

dense_test_tfidf_with_linguistic = dense_test_tfidf_with_linguistic.toarray()  # Convert to dense array

# Evaluate the model using the test data
test_loss, test_accuracy = model.evaluate(dense_test_tfidf_with_linguistic, test_labels)
print(f'Test accuracy: {test_accuracy}')

191/191 [==============================] - 22s 103ms/step - loss: 0.2277 - accuracy: 0.9076
Test accuracy: 0.9075809717178345


In [24]:
model.save('bi_lstm_tfidf.h5')

In [25]:
test_accuracy * lengthTestData

5518.999889016151